# Machine learning - Assignment 4 - Classification
____
**Author**: Kemal Cikota

**Course**: Machine learning
____

## Introduction

In tis fuckin assinemt type sit.

## Conceptual Questions

ANswers to conceptual questions.

## Practical

Short intro about what is about to happen.

### Load the data and get an overview of the data

In [1]:
import pandas as pd # Never coded in R before but this seems to be the equivalent of library(pandas) in R
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

# load Smarket.csv
boston = pd.read_csv('Smarket.csv')

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)